In [1]:
import argparse
import json
import sys
import time
import socket
import pandas as pd
from confluent_kafka import Consumer, KafkaError, KafkaException

In [2]:
from ksql import KSQLAPI
import socket
import pandas as pd
import matplotlib.pyplot as plt
# import mysql.connector as sql
import time
import copy
import datetime
from datetime import timedelta
import psycopg2
import pandas.io.sql as psql
import os
from datetime import datetime
from multiprocessing import Pool, freeze_support
import numpy as np

import logging
logging.basicConfig(
            format='%(asctime)s %(levelname)-8s %(message)s',
            level=logging.INFO,
            datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)
import multiprocessing as mp

In [3]:
import json
import re
import pandas as pd
import time

In [4]:
from ksql_data_helpers import return_det_counts_data, return_recent_data_ts,ts_str_to_unix, ts_unix_to_ts

In [6]:
from helpers import return_det_waveform

In [57]:
isc_eoi = IscEoiStore()

In [58]:
all_rows = isc_eoi.start_processing()

  SIGNALID    WINDOWSTART      WINDOWEND  SUMFLAG  processed
0     1010  1672703700000  1672705200000       25          1
1     1015  1672703700000  1672705200000       25          1
2     1873  1672703700000  1672705200000       25          1
4     2565  1672703700000  1672705200000       25          1
allrows
[SIGNALID                1010
WINDOWSTART    1672703700000
WINDOWEND      1672705200000
SUMFLAG                   25
processed                  1
Name: 0, dtype: object, SIGNALID                1015
WINDOWSTART    1672703700000
WINDOWEND      1672705200000
SUMFLAG                   25
processed                  1
Name: 1, dtype: object, SIGNALID                1873
WINDOWSTART    1672703700000
WINDOWEND      1672705200000
SUMFLAG                   25
processed                  1
Name: 2, dtype: object, SIGNALID                2565
WINDOWSTART    1672703700000
WINDOWEND      1672705200000
SUMFLAG                   25
processed                  1
Name: 4, dtype: object]
redu {'det

In [59]:
all_rows

[('1010',
  {10103: {'detector_id': '10103',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 3,
    'curr_ma': 12.75,
    'baseline': 12.0,
    'reduction': 0.75},
   10109: 'None',
   101010: 'None'}),
 ('1015',
  {10152: 'None',
   10157: {'detector_id': '10157',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 8,
    'curr_ma': 8.75,
    'baseline': 8.75,
    'reduction': 0.08571428571428572},
   10158: 'None'}),
 ('1873', {}),
 ('2565',
  {25654: {'detector_id': '25654',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 23,
    'curr_ma': 24.75,
    'baseline': 30.35,
    'reduction': 0.24217462932454697},
   25655: {'detector_id': '25655',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 21,
    'curr_ma': 27.0,
    'baseline': 31.25,
    'reduction': 0.328},
   256512: {'detector_id': '256512',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 27,
    'curr_ma': 26.25,
    'baseline': 28.4,
    'reduct

In [49]:
p = mp.Pool(isc_eoi.no_of_cores)
results = p.map(isc_eoi.process_single_isc, all_rows)
p.close()

redu {'detector_id': '25654', 'end_time': '2023-01-02T19:24:59.999000', 'curr_volume': 22, 'curr_ma': 23.5, 'baseline': 29.1, 'reduction': 0.2439862542955327}
redu {'detector_id': '25655', 'end_time': '2023-01-02T19:24:59.999000', 'curr_volume': 16, 'curr_ma': 22.0, 'baseline': 28.7, 'reduction': 0.4425087108013937}
redu {'detector_id': '256512', 'end_time': '2023-01-02T19:24:59.999000', 'curr_volume': 24, 'curr_ma': 27.0, 'baseline': 27.65, 'reduction': 0.13200723327305602}
redu {'detector_id': '256513', 'end_time': '2023-01-02T19:24:59.999000', 'curr_volume': 29, 'curr_ma': 31.0, 'baseline': 34.55, 'reduction': 0.16063675832127344}


In [50]:
for isc_id, each_result in results:
    if(isc_id not in isc_eoi.all_isc_store ): # isc is not yet in store,
        isc_eoi.all_isc_store[isc_id]={} # dict for storing all dets
        for det_id, det_value in each_result.items():
            if(det_value != 'None'): # if reduction exist
                isc_eoi.all_isc_store[isc_id][det_id]=[det_value]
            else:
                isc_eoi.all_isc_store[isc_id][det_id]=[]

    else:
        for det_id, det_value in each_result.items():
            if(det_id not in isc_eoi.all_isc_store[isc_id]):
                if(det_value != 'None'):
                    isc_eoi.all_isc_store[isc_id][det_id]=[det_value]
                else:
                    isc_eoi.all_isc_store[isc_id][det_id] = []
            else:
                if(det_value != 'None'):
                    isc_eoi.all_isc_store[isc_id][det_id].append(det_value)
                else:
                    isc_eoi.all_isc_store[isc_id][det_id] = []

In [51]:
isc_eoi.all_isc_store

{'1010': {10103: [],
  10109: [{'detector_id': '10109',
    'end_time': '2023-01-02T19:24:59.999000',
    'curr_volume': 12,
    'curr_ma': 12.5,
    'baseline': 15.1,
    'reduction': 0.2052980132450331}],
  101010: []},
 '1015': {10152: [],
  10157: [{'detector_id': '10157',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 8,
    'curr_ma': 8.75,
    'baseline': 8.75,
    'reduction': 0.08571428571428572},
   {'detector_id': '10157',
    'end_time': '2023-01-02T19:24:59.999000',
    'curr_volume': 6,
    'curr_ma': 7.25,
    'baseline': 8.333333333333334,
    'reduction': 0.28}],
  10158: [{'detector_id': '10158',
    'end_time': '2023-01-02T19:24:59.999000',
    'curr_volume': 7,
    'curr_ma': 8.5,
    'baseline': 8.5,
    'reduction': 0.17647058823529413}]},
 '1873': {},
 '2565': {25654: [{'detector_id': '25654',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 23,
    'curr_ma': 24.75,
    'baseline': 30.35,
    'reduction': 0.24217462932454697},
 

In [61]:
isc_eoi.all_isc_store

{'1010': {10103: [{'detector_id': '10103',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 3,
    'curr_ma': 12.75,
    'baseline': 12.0,
    'reduction': 0.75}],
  10109: [],
  101010: []},
 '1015': {10152: [],
  10157: [{'detector_id': '10157',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 8,
    'curr_ma': 8.75,
    'baseline': 8.75,
    'reduction': 0.08571428571428572}],
  10158: []},
 '1873': {},
 '2565': {25654: [{'detector_id': '25654',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 23,
    'curr_ma': 24.75,
    'baseline': 30.35,
    'reduction': 0.24217462932454697}],
  25655: [{'detector_id': '25655',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 21,
    'curr_ma': 27.0,
    'baseline': 31.25,
    'reduction': 0.328}],
  256512: [{'detector_id': '256512',
    'end_time': '2023-01-02T19:19:59.999000',
    'curr_volume': 27,
    'curr_ma': 26.25,
    'baseline': 28.4,
    'reduction': 0.04929577464788728}

In [30]:
df_filtered = isc_eoi.get_recent_status()

In [31]:
df_filtered

,SIGNALID,WINDOWSTART,WINDOWEND,SUMFLAG,processed
0,1010,1672703100000,1672704600000,25,1
3,1015,1672703100000,1672704600000,25,1
6,1873,1672703100000,1672704600000,25,1
12,2565,1672703100000,1672704600000,25,1


In [60]:
isc_eoi.isc_last_processed 

{'1010': 1672705200000,
 '1015': 1672705200000,
 '1873': 1672705200000,
 '2565': 1672705200000}

In [55]:
class IscEoiStore():
    
    def __init__(self):
        self.all_isc_store = {}
        self.isc_last_processed = {}
        self.map_file = pd.read_csv('MAPPINGS_DET_INFO_OCT_2022.csv')
        self.reduction_threshold = 0.01
        self.volume_threshold = 1
        self.no_of_cores = 30
        
    def process_logic(self,signalid, window_end):
        if(signalid not in self.isc_last_processed):
            return 1
        elif(window_end > self.isc_last_processed[signalid]):
            return 1
        return 0
            
        
    def get_recent_status(self):
        df = return_recent_data_ts(look_back_time = 15)
        
        df_filtered = df[df.SUMFLAG==25].copy()
        df_filtered['processed'] = df_filtered.apply(lambda x : self.process_logic(x.SIGNALID, x.WINDOWEND), axis =1 )
        df_filtered = df_filtered[df_filtered['processed']==1]
        df_filtered = df_filtered.drop_duplicates(['SIGNALID'], keep='first')
        
        return df_filtered
    
    def get_detector_ids(self, signalid):
        all_dets = self.map_file[self.map_file.ATSPM_ID.isin([signalid])]
        dets_of_interest = all_dets[(all_dets.phase.isin(['2','6'])) & (all_dets.distanceToStopbar >150) ]
        # storing all the detector ID's. Detector ID is SignalID + Channel no.
        detector_ids = []
        for i, each_row in dets_of_interest.iterrows():
            each_signal = each_row['ATSPM_ID']
            channel = each_row['channel']
            detector_ids.append(int(str(each_signal)+ str(channel)))
        return detector_ids

    def process_single_isc(self, row):
        isc_result = {}
        
        all_dets = self.get_detector_ids(row.SIGNALID)
        
        for each_det in all_dets:
            le = LabelEOI(each_det, row.WINDOWSTART, row.WINDOWEND)
            reduction_flag, reduction_dict = le.get_EOI()
            if(reduction_flag):
                print(f"redu {reduction_dict}")
                if((reduction_dict['reduction']>self.reduction_threshold) and(reduction_dict['curr_ma']>self.volume_threshold)):
                    isc_result[each_det]= reduction_dict
            else:
                isc_result[each_det] = 'None'
        return row.SIGNALID, isc_result
                    
    def start_processing(self):
#         get current data
        df_recent = self.get_recent_status()
        print(df_recent)
#     filter if next timestamp is available
        all_rows = []
    
        for i, each_row in df_recent.iterrows():
            if(each_row.SIGNALID not in self.isc_last_processed ):
                all_rows.append(each_row)
                self.isc_last_processed[each_row.SIGNALID] = each_row.WINDOWEND
            else:
                if(each_row.WINDOWEND>self.isc_last_processed[each_row.SIGNALID]):
#                     new timestamp available
                    all_rows.append(each_row)
                    self.isc_last_processed[each_row.SIGNALID] = each_row.WINDOWEND
#         return all_rows
#     parallel process all rows
        print(f"allrows")
        print(all_rows)
        p = mp.Pool(self.no_of_cores)
        results = p.map(self.process_single_isc, all_rows)
        p.close()
        
        for isc_id, each_result in results:
            if(isc_id not in self.all_isc_store ): # isc is not yet in store,
                self.all_isc_store[isc_id]={} # dict for storing all dets
                for det_id, det_value in each_result.items():
                    if(det_value != 'None'): # if reduction exist
                        self.all_isc_store[isc_id][det_id]=[det_value]
                    else:
                        self.all_isc_store[isc_id][det_id]=[]
                        
            else:
                for det_id, det_value in each_result.items():
                    if(det_id not in self.all_isc_store[isc_id]):
                        if(det_value != 'None'):
                            self.all_isc_store[isc_id][det_id]=[det_value]
                        else:
                            self.all_isc_store[isc_id][det_id] = []
                    else:
                        if(det_value != 'None'):
                            self.all_isc_store[isc_id][det_id].append(det_value)
                        else:
                            self.all_isc_store[isc_id][det_id] = []
                            
                
            
        
        return results


            


In [56]:
class LabelEOI():
    
    
    
    def __init__(self, detector_id, start_time, end_time):
        super().__init__()
        self.no_of_cores = 30
        self.aggregation_level = 300
        self.min_prev_data = 2
        self.window_size = 4
        self.detector_id =str(detector_id)
        self.start_time= start_time
        self.end_time = end_time-1 


        # ALL THRESHOLDS  AND CONSTANTS
        self.DET_HISTORY_VOLUME_THRESHOLD = 10
        self.time_start = '07:00'
        self.time_end = '21:00'
        self.prev_days_to_consider = [7,14,21]
        self.current_weight = 2
        self.history_weight = 1

        
    def compute_historic_baseline(self, row):

        """
        Computes historic baseline traffic for a given time interval
        Parameters
        ----------
        row : dataframe row
            More info to be displayed (default is None)

        Returns
        -------
        value: float
            traffic volume baseline
        """
        all_prev = ['MA_prev_7','MA_prev_14','MA_prev_21' ]
        value = 0
        count = self.current_weight
        value += self.current_weight*row['MA_curr']
        for prev in all_prev:
            if(prev in row and row[prev]>self.DET_HISTORY_VOLUME_THRESHOLD):
                value+=self.history_weight*row[prev]
                count+=self.history_weight
                
        return value/count
    
    def get_EOI(self):
        flag, hist_df = self.return_hist_df()
        if(flag):
            last_row = hist_df.iloc[-1]
            curr_vol = last_row.Count
            curr_ma = last_row.MA_curr
            reduction =  (last_row.baseline- curr_vol)/(last_row.baseline)
            reducton_dict = {'detector_id':self.detector_id, 'end_time':ts_unix_to_ts(self.end_time), 'curr_volume':curr_vol, 'curr_ma':curr_ma,'baseline':last_row.baseline, 'reduction':reduction}
            if(reduction>0):
                return True, reducton_dict
            else:
                return False, 0
        return False, 0
            
    
    
    def return_hist_df(self):
        # current data
        count_df = return_det_counts_data(self.detector_id , self.start_time, self.end_time)
#         count_df.rename(columns={'EventCode': 'Count_curr'}, inplace=True)

        count_df.rename(columns={'COUNT': 'Count'}, inplace=True)
        ll = f"{self.aggregation_level}s"
        count_df= count_df.resample(ll).sum()
        count_df['MA_curr'] = count_df.rolling(window=self.window_size ).mean()
        count_df.fillna(method='bfill', inplace = True)
        count_df['time'] = count_df.index.time
        
        
        # history from previous weeks


        prev_count ={}
        all_prev = [7,14, 21]
        for i in  all_prev:
            start_time_prev =  (pd.to_datetime(ts_unix_to_ts(self.start_time)) - pd.Timedelta(days=i)).strftime("%Y-%m-%d %H:%M:%S")
            end_time_prev =  (pd.to_datetime(ts_unix_to_ts(self.end_time)) - pd.Timedelta(days=i)).strftime("%Y-%m-%d %H:%M:%S")
            # print(f"time prev {time_prev}")

            count_df_prev = return_det_waveform(self.detector_id ,start_time_prev, end_time_prev)
#             print(f"count_df_prev shape {count_df_prev.shape}")
            if(count_df_prev.shape[0]<5):
                pass
            else:
        #         count_df_prev.drop(columns = ['SignalID','EventParam'], inplace = True)
        #         count_df_prev.rename(columns={'EventCode': 'Count'}, inplace=True)
                ll = f"{self.aggregation_level}s"
                count_df_prev= count_df_prev.resample(ll).sum()
                count_df_prev[f'MA_prev_{i}'] = count_df_prev.rolling(window=self.window_size).mean()
                count_df_prev[f'time'] =  count_df_prev.index.time
                count_df_prev.fillna(method='bfill', inplace = True)
                prev_count[i]  = count_df_prev  
                
                
        all_merged_df =   count_df.copy() 
        all_merged_df['timestamp'] = all_merged_df.index
        #         print(f"Yash DEBUG   all_merged_df {all_merged_df.head(5)}" )
        
#         print(f"prev count {len(prev_count)}")
        if(len(prev_count) > self.min_prev_data):
            for k,each_prev in   prev_count.items():
                all_merged_df = pd.merge(all_merged_df,each_prev, on = 'time')
                
            all_merged_df['baseline'] = all_merged_df.apply(lambda x: self.compute_historic_baseline(x), axis = 1) 
            all_merged_df['DetectorID'] = self.detector_id
            all_merged_df.set_index('timestamp', inplace = True)
            
            return True, all_merged_df
        else:
            return False, None

    
    
        
    


In [4]:
client = KSQLAPI('http://localhost:8088')
streamProperties = {
    "ksql.streams.auto.offset.reset": "earliest", 
    "ksql.query.pull.table.scan.enabled": "true"
}

In [6]:
def parse_columns(columns_str):
    regex = r"(?<!\<)`(?P<name>[A-Z_]+)` (?P<type>[A-z]+)[\<, \"](?!\>)"
    result = []

    matches = re.finditer(regex, columns_str)
    for matchNum, match in enumerate(matches, start=1):
        result.append({"name": match.group("name"), "type": match.group("type")})

    return result


def process_row(row, column_names):
    row = row.replace(",\n", "").replace("]\n", "").rstrip("]")
    row_obj = json.loads(row)
    if "finalMessage" in row_obj:
        return None
    column_values = row_obj["row"]["columns"]
    index = 0
    result = {}
    for column in column_values:
        result[column_names[index]["name"]] = column
        index += 1

    return result

def process_query_result(results, return_objects=None):
    if return_objects is None:
        yield from results

    # parse rows into objects
    try:
        header = next(results)
    except StopIteration:
        return
    columns = parse_columns(header)

    for result in results:
        row_obj = process_row(result, columns)
        if row_obj is None:
            return
        yield row_obj
        
def return_query_df(ksql_string):
    query = client.query(ksql_string, stream_properties=streamProperties)
    tspm = list(process_query_result(query, return_objects=True))
    return pd.DataFrame(tspm)
 

def ts_unix_to_ts(time_unix):
    dt = datetime.fromtimestamp(time_unix/1000)
    return dt.strftime('%Y-%m-%dT%H:%M:%S.%f')    
    
def ts_str_to_unix(time_str):
    dt = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S.%f')
    return int(time.mktime(dt.timetuple()) * 1000)

In [8]:
# FUNCTION TO RETURN MOST RECENT BIT MASK FOR ALL INTERSECTIONS
def return_recent_data_ts(look_back_time = 15):
    '''
    look_back_time: time in min 
    '''
    current_time_ms= round(time.time()*1000)
    look_time = current_time_ms - look_back_time*60*1000
    TABLE_NAME = 'ATSPM_MASK_1500SEC' #THIS IS A TABLE
    ksql_string = f'SELECT *  from {TABLE_NAME} where WINDOWEND >=' + f'{look_time}' 
    print(ksql_string)
    df = return_query_df(ksql_string)
    return df


# FUNCTION TO RETURN COUNTS FOR A DETECTOR BETWEEN START AND END TIME. 5 sec aggregation
def return_det_counts_data(detector_id, time_start, time_end):
    '''
    detector_id: detector_id string (signalID + channel value)
    start_time: unix time in ms (INT)
    end_time: unix time in ms(INT)
    '''
    TABLE_NAME = 'COUNTS_RAW_STREAM_V1_AGG_5SEC' #Detector ID to counts  #THIS IS A TABLE
    ksql_string = f'SELECT * FROM {TABLE_NAME} where DETECTORID = ' +  "'" + detector_id + "'"   + f' AND WINDOWSTART>={time_start} AND WINDOWSTART < {time_end}'
    print(ksql_string)
    temp_df = return_query_df(ksql_string)
    
    temp_df.drop(columns = ['DETECTORID', 'WINDOWEND'], inplace = True)
    new_row = pd.Series(data={'WINDOWSTART':time_start,'COUNT':0}, name=0)
    temp_df = temp_df.append(new_row, ignore_index=False)
    new_row = pd.Series(data={'WINDOWSTART':time_end,'COUNT':0}, name=0)
    temp_df = temp_df.append(new_row, ignore_index=False)
    temp_df['timestamp']  = pd.to_datetime(temp_df['WINDOWSTART'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    temp_df.set_index('timestamp', inplace = True)
    temp_df.drop(columns = ['WINDOWSTART'], inplace = True)
    temp_df.sort_index(inplace = True)

    final_df = temp_df.resample('5s').sum()
    
    
    return final_df

SELECT *  from ATSPM_MASK_1500SEC where WINDOWEND >=1672283807888


In [10]:
df_filtered = df[df.SUMFLAG==25]

2022-12-28 22:32:00 INFO     Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-12-28 22:32:00 INFO     NumExpr defaulting to 8 threads.


In [11]:
df_filtered

,SIGNALID,WINDOWSTART,WINDOWEND,SUMFLAG
0,1010,1672282500000,1672284000000,25
1,1010,1672282800000,1672284300000,25
7,1015,1672282500000,1672284000000,25
8,1015,1672282800000,1672284300000,25
15,1873,1672282500000,1672284000000,25
16,1873,1672282800000,1672284300000,25
22,2565,1672282500000,1672284000000,25
23,2565,1672282800000,1672284300000,25


In [12]:
df_filtered = df_filtered.drop_duplicates(['SIGNALID'], keep='last')

In [13]:
df_filtered

,SIGNALID,WINDOWSTART,WINDOWEND,SUMFLAG
1,1010,1672282800000,1672284300000,25
8,1015,1672282800000,1672284300000,25
16,1873,1672282800000,1672284300000,25
23,2565,1672282800000,1672284300000,25


In [14]:
curr_row = df_filtered.iloc[0]

In [15]:
all_dets = map_file[map_file.ATSPM_ID.isin([curr_row.SIGNALID])]
dets_of_interest = all_dets[(all_dets.phase.isin(['2','6'])) & (all_dets.distanceToStopbar >150) ]

# storing all the detector ID's. Detector ID is SignalID + Channel no.
detector_ids = []
for i, each_row in dets_of_interest.iterrows():
    each_signal = each_row['ATSPM_ID']
    channel = each_row['channel']
    detector_ids.append(int(str(each_signal)+ str(channel)))

In [16]:
detector_ids

[10103, 10109, 101010]

In [17]:
curr_row

SIGNALID                1010
WINDOWSTART    1672282800000
WINDOWEND      1672284300000
SUMFLAG                   25
Name: 1, dtype: object

In [19]:
# for one detector 

det_id = str(detector_ids[1])

# current data
count_df = return_det_counts_data(det_id, curr_row.WINDOWSTART, curr_row.WINDOWEND-1)
count_df.rename(columns={'EventCode': 'Count_curr'}, inplace=True)

count_df.rename(columns={'COUNT': 'Count_curr'}, inplace=True)
ll = f"300s"
count_df= count_df.resample(ll).sum()
count_df['MA_curr'] = count_df.rolling(window=4 ).mean()
count_df.fillna(method='bfill', inplace = True)
count_df['time'] = count_df.index.time


SELECT * FROM COUNTS_RAW_STREAM_V1_AGG_5SEC where DETECTORID = '10109' AND WINDOWSTART>=1672282800000 AND WINDOWSTART < 1672284299999


In [26]:
# history from previous weeks


start_time= ts_unix_to_ts(curr_row.WINDOWSTART)
end_time = ts_unix_to_ts(curr_row.WINDOWEND-1)
prev_count ={}
all_prev = [7,14, 21]
for i in  all_prev:
    start_time_prev =  (pd.to_datetime(start_time) - pd.Timedelta(days=i)).strftime("%Y-%m-%d %H:%M:%S")
    end_time_prev =  (pd.to_datetime(end_time) - pd.Timedelta(days=i)).strftime("%Y-%m-%d %H:%M:%S")
    # print(f"time prev {time_prev}")

    count_df_prev = return_det_waveform(det_id,start_time_prev, end_time_prev)
    # print(f"count_df_prev shape {count_df_prev.shape}")
    if(count_df_prev.shape[0]<5):
        pass
    else:
#         count_df_prev.drop(columns = ['SignalID','EventParam'], inplace = True)
#         count_df_prev.rename(columns={'EventCode': 'Count'}, inplace=True)
        ll = f"300s"
        count_df_prev= count_df_prev.resample(ll).sum()
        count_df_prev[f'MA_prev_{i}'] = count_df_prev.rolling(window=4).mean()
        count_df_prev[f'time'] =  count_df_prev.index.time
        count_df_prev.fillna(method='bfill', inplace = True)
        prev_count[i]  = count_df_prev  

In [31]:
all_merged_df =   count_df.copy() 
all_merged_df['timestamp'] = all_merged_df.index
#         print(f"Yash DEBUG   all_merged_df {all_merged_df.head(5)}" )
if(len(prev_count) > 2):
    for k,each_prev in   prev_count.items():
#                 all_merged_df = pd.merge(all_merged_df,each_prev, on = 'time', right_index= True)
        all_merged_df = pd.merge(all_merged_df,each_prev, on = 'time')
#                 print(f"Yash DEBUG   all_merged_df  last {all_merged_df.head(5)}" )
    all_merged_df['baseline'] = all_merged_df.apply(lambda x: compute_historic_baseline(x), axis = 1)
    all_merged_df['DetectorID'] = det_id
    all_merged_df.set_index('timestamp', inplace = True)

In [32]:
all_merged_df

,Count_curr_x,MA_curr,time,Count_curr_y,MA_prev_7,Count_curr_x,MA_prev_14,Count_curr_y,MA_prev_21,baseline,DetectorID
timestamp,,,,,,,,,,,
2022-12-28 22:00:00-05:00,14,10.5,22:00:00,8,14.00,1,3.00,3,4.5,11.666667,10109
2022-12-28 22:05:00-05:00,15,10.5,22:05:00,10,14.00,3,3.00,8,4.5,11.666667,10109
2022-12-28 22:10:00-05:00,6,10.5,22:10:00,20,14.00,5,3.00,2,4.5,11.666667,10109
2022-12-28 22:15:00-05:00,7,10.5,22:15:00,18,14.00,3,3.00,5,4.5,11.666667,10109
2022-12-28 22:20:00-05:00,10,9.5,22:20:00,7,13.75,0,2.75,7,5.5,10.916667,10109


In [30]:
 def compute_historic_baseline(row):

        all_prev = ['MA_prev_7','MA_prev_14','MA_prev_21' ]
        value = 0
        count = 2
        value += 2*row['MA_curr']
        for prev in all_prev:
            if(prev in row and row[prev]>10):
                value+=1*row[prev]
                count+=1
                
        return value/count

In [99]:
count_df

,Count_curr,MA_curr,time
timestamp,,,
2022-12-28 19:20:00-05:00,25,21.0,19:20:00
2022-12-28 19:25:00-05:00,20,21.0,19:25:00
2022-12-28 19:30:00-05:00,23,21.0,19:30:00
2022-12-28 19:35:00-05:00,16,21.0,19:35:00
2022-12-28 19:40:00-05:00,23,20.5,19:40:00


In [ ]:
msg.row

In [26]:
help(msg.value())

Help on bytes object:

class bytes(object)
 |  bytes(iterable_of_ints) -> bytes
 |  bytes(string, encoding[, errors]) -> bytes
 |  bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
 |  bytes(int) -> bytes object of size given by the parameter initialized with null bytes
 |  bytes() -> empty bytes object
 |  
 |  Construct an immutable array of bytes from:
 |    - an iterable yielding integers in range(256)
 |    - a text string encoded using the specified encoding
 |    - any object implementing the buffer API.
 |    - an integer
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(...)
 |  
 |  __g

In [14]:
help(msg)

Help on Message object:

class Message(builtins.object)
 |  The Message object represents either a single consumed or produced message, or an event (:py:func:`error()` is not None).
 |  
 |  An application must check with :py:func:`error()` to see if the object is a proper message (error() returns None) or an error/event.
 |  
 |  This class is not user-instantiable.
 |  
 |  .. py:function:: len()
 |  
 |    :returns: Message value (payload) size in bytes
 |    :rtype: int
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __len__(self, /)
 |      Return len(self).
 |  
 |  error(...)
 |      The message object is also used to propagate errors and events, an application must check error() to determine if the Message is a proper message (error() returns None) or an error or event (error() returns a KafkaError object)
 |      
 |      :rtype: None or :py:class:`KafkaError`
 |  
 |  headers(...)
 |      Retrieve the header

In [7]:
msg

In [8]:
val = msg.value()

In [9]:
val

b'{"SUMFLAG":1}'

In [11]:
dval = json.loads(val)

In [12]:
dval

{'SUMFLAG': 1}